In [13]:
import sys
import networkx
import pandas as pd
import numpy as np
import matplotlib.pyplot
import random
import math
import itertools
from scipy.special import gammaln

In [14]:
inputfilename = 'data//example.csv'
outputfilename = 'example.gph'

df = pd.read_csv(inputfilename)
bayes_net = networkx.DiGraph() #create the bayes net
nodes = df.columns
bayes_net.add_nodes_from(df.columns) #add the columns as nodes to the bayes net

def export_connections(bayes_net, filename):
   
    with open(filename, 'w') as f: 
        for parent, child in bayes_net.edges():
            f.write(f"{parent},{child}\n") #write to the file in the format required

In [18]:
def compute_node_bayesian_score(node,bayes_net,data):
    node_score = 0
    parents = list(bayes_net.predecessors(node)) # determine which nodes are parents of the current node
    parent_ri = df[parents].nunique() # get how many possible parents there are in a list
    qi = np.prod(parent_ri) # multiply the possible parents to get the total number of possible parential instantiations
    ri = data[node].nunique() # determine number of unique values of current node
    alpha_ij0 = ri
    alpha_ijk = 1 #uniform prior

    possible_parent_values = [data[parent].unique().tolist() for parent in parents] #get all values of possible parents
    possible_parent_combinations = itertools.product(*possible_parent_values) #find the unique combinations of the parent values
    
    for parent_combination in possible_parent_combinations: #second summation in the equation
        compare_logic = (data[parents].eq(parent_combination).all(axis=1)) #compare the parent combination to the data
        m_ij0 = len(data[compare_logic]) #determine how many match

        node_score = node_score + gammaln(alpha_ij0) - gammaln(alpha_ij0 + m_ij0) # Algorithms for Decision Making section 5.1

        for k in range(ri): # 3rd summation in the equation
            m_ijk = len(data[compare_logic & (data[node] == (k + 1))]) # compute where parent values and node values match
            node_score = node_score + gammaln((alpha_ijk+m_ijk)/alpha_ijk) # Algorithms for Decision Making section 5.1


    return node_score

def bayes_score_func(bayes_net, df):
    nodelist = list(bayes_net.nodes)
    bayesian_score = 0
    for node in nodelist:
        node_score = compute_node_bayesian_score(node, bayes_net, df)
        bayesian_score = bayesian_score + node_score #first summation in the equation

    return bayesian_score


-132.57689402451837


In [ ]:

def AddLimitedK2Chain(bayes_net, nodelist, maximum_edges=3):
    lengthNodeList = lengthNodeList
    for i in range(lengthNodeList):
        current_node = nodelist[i]
        edges_added = 0  # counter for connections added
        for j in range(i + 1, lengthNodeList): #i+1 to end since we can only connect to the next node and beyond
            if edges_added >= maximum_edges:
                break  # stop adding edges if the limit is reached
            bayes_net.add_edge(current_node, nodelist[j]) # jth node away from i 
            edges_added = edges_added + 1  # increment the counter
    return bayes_net

def RandomSwaps(nodelist,num_swaps,num_options):

    permutations = []  # list to store the permutations

    for _ in range(num_options):
            swapped_list = nodelist.copy()
            length = len(swapped_list)
            for i in range(num_swaps):
                [index1, index2] = random.sample(range(length), 2) # choose two random indexes
                swapped_list[index1], swapped_list[index2] = swapped_list[index2], swapped_list[index1] #swap around
            permutations.append(swapped_list)
    return permutations
    
def Hill_climb_initalization(bayes_net,df):
    nodelist = list(bayes_net.nodes)
    random.shuffle(nodelist)  # initialize random order
    print("Initial Node List:", nodelist)

    AddLimitedK2Chain(bayes_net, nodelist) # add the limited K2 chain
    best_score = bayes_score_func(bayes_net, df)
    print("Initial Score:", best_score)

    # Start the hill climbing loop
    while True:
        total_scores = []
        num_options = 10
        num_swaps = 10
        permutations = RandomSwaps(nodelist, num_swaps, num_options) # do the random swap
        
        for i in range(num_options):
            bayes_net = networkx.DiGraph() #create new temporary bayes net
            bayes_net.add_nodes_from(permutations[i])  # add nodes 
            AddLimitedK2Chain(bayes_net, permutations[i])  #add edges
            
            score = bayes_score_func(bayes_net, df) #compute score
            print(f"Score for permutation {i + 1}: {score}")
            total_scores.append(score) #add the score to the list 
        
        best_permutation_index = np.argmax(total_scores) #find the best index
        new_best_score = total_scores[best_permutation_index] #find the best score
        print(f'Best score this iteration: {new_best_score}')
        
        if new_best_score < best_score:
            print("No improvement found")
            break  

        best_score = new_best_score
        nodelist = permutations[best_permutation_index]  # update the nodelist with the new best

    print(f'New Best Node List: {nodelist}')

    
    return nodelist

def k2_search(nodelist, bayes_net, data):

    bayes_net.remove_edges_from(list(bayes_net.edges())) # reset the bayes net

    for i, node in enumerate(nodelist):
        current_parents = list(bayes_net.predecessors(node))
        print(f"Current node: {node}")
        best_score = bayes_score_func(bayes_net, data) # get a baseline bayes score

        for j in range(i + 1, len(nodelist)): 
            possible_child = nodelist[j]
            bayes_net_temp = bayes_net.copy()
            bayes_net_temp.add_edge(possible_child, node) #new bayes net with possible parent

            score = bayes_score_func(bayes_net_temp, data)
            print(f"Child: {possible_child} Score: {score}")

            if score > best_score: #if netter score
                print(f"Adding {possible_child} as a child of {node}")
                bayes_net.add_edge(possible_child, node) #update the master bayes net
                best_score = score  # update the best score

            if len(list(bayes_net.predecessors(node))) >= 4: #check if there are already 4
                print(f"Max parents reached, moving on")
                break 

    return bayes_net

In [ ]:

nodelist = Hill_climb_initalization(bayes_net,df) # perform hill climb
bayes_net = k2_search(nodelist, bayes_net, df) #perofrm K2
export_connections(bayes_net, outputfilename) #export for graphinh